# 🚀 GPU-Accelerated Song Release Year Prediction - Model Training (RAPIDS)

This notebook trains multiple regression models using **RAPIDS cuML** for GPU acceleration where available, with CPU fallback for unsupported models.

**Dataset Context:** Audio features of songs (timbre, pitch, rhythm, etc.) used to predict release year.

**RAPIDS cuML Benefits:**
- 🔥 10-100x faster model training on GPU
- ⚡ Same API as scikit-learn
- ? Automatic CPU fallback for unsupported models

## Models to Train:
1. **Linear Regression** (cuML GPU or sklearn CPU)
2. **Ridge Regression** (cuML GPU or sklearn CPU)
3. **Lasso Regression** (cuML GPU or sklearn CPU)
4. **ElasticNet Regression** (cuML GPU or sklearn CPU)
5. **Decision Tree Regressor** (sklearn CPU - no GPU version)
6. **Random Forest Regressor** (cuML GPU or sklearn CPU)
7. **Gradient Boosting Regressor** (sklearn CPU - no GPU version)
8. **XGBoost Regressor** (GPU hist or CPU - if available)
9. **Support Vector Regressor** (sklearn CPU - no GPU version)
10. **K-Nearest Neighbors Regressor** (cuML GPU or sklearn CPU)

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
from datetime import datetime
import time

# Check GPU availability and import RAPIDS
print("="*80)
print("GPU AVAILABILITY CHECK")
print("="*80)

try:
    import cupy as cp
    import cudf
    from cuml.model_selection import train_test_split as cu_train_test_split
    from cuml.linear_model import LinearRegression as cuLinearRegression
    from cuml.linear_model import Ridge as cuRidge
    from cuml.linear_model import Lasso as cuLasso
    from cuml.linear_model import ElasticNet as cuElasticNet
    from cuml.ensemble import RandomForestRegressor as cuRandomForestRegressor
    from cuml.neighbors import KNeighborsRegressor as cuKNeighborsRegressor
    from cuml.metrics import mean_squared_error as cu_mse
    from cuml.metrics import r2_score as cu_r2_score
    
    rapids_available = True
    print("✓ RAPIDS cuML available")
    gpu_count = cp.cuda.runtime.getDeviceCount()
    print(f"✓ GPUs available: {gpu_count}")
    
    if gpu_count > 0:
        gpu_name = cp.cuda.runtime.getDeviceProperties(0)['name'].decode()
        gpu_mem = cp.cuda.runtime.getDeviceProperties(0)['totalGlobalMem'] / 1e9
        print(f"✓ GPU 0: {gpu_name}")
        print(f"✓ GPU Memory: {gpu_mem:.1f} GB")
        
except ImportError:
    rapids_available = False
    print("❌ RAPIDS not available")
    print("\n📦 Installation: conda install -c rapidsai -c conda-forge -c nvidia rapids")
    print("\nFalling back to CPU training with scikit-learn...")

# Standard sklearn imports (for CPU fallback and non-GPU models)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Try to import XGBoost
try:
    import xgboost as xgb
    xgboost_available = True
    print("✓ XGBoost available")
except ImportError:
    xgboost_available = False
    print("⚠ XGBoost not available")

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')

print("\n✓ All libraries imported successfully!")
print(f"🚀 GPU Acceleration: {'ENABLED' if rapids_available else 'DISABLED (CPU mode)'}")
print("="*80)

## Load Processed Song Features Data (GPU)

In [ ]:
# Load the processed and scaled data directly to GPU
print("Loading processed song features data to GPU...\n")

# Load data with cuDF (directly to GPU)
gdf = cudf.read_csv('cars_scaled_standard_rapids.csv')

print(f"✓ Data loaded to GPU: {gdf.shape}")
print(f"Columns (audio features): {gdf.shape[1]}")
print(f"Rows (songs): {gdf.shape[0]:,}")

# Display first few rows
print("\nFirst few rows:")
gdf.head()

## Prepare Features and Target (GPU)

In [ ]:
# Separate features and target
target_col = gdf.columns[0]
X = gdf.drop(columns=[target_col])
y = gdf[target_col]

print(f"Target variable: {target_col} (Song Release Year)")
print(f"Number of features: {X.shape[1]}")
print(f"Number of samples: {X.shape[0]:,}")
print(f"\nTarget statistics:")
print(y.describe())

## Train-Test Split (GPU)

In [ ]:
# Split data using cuML (on GPU)
print("Splitting data into train and test sets on GPU...\n")

X_train, X_test, y_train, y_test = cu_train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    shuffle=True
)

print(f"Training set: {X_train.shape[0]:,} songs")
print(f"Test set: {X_test.shape[0]:,} songs")
print(f"Features: {X_train.shape[1]}")

# Save test indices for later evaluation
test_indices = X_test.index.to_pandas()
with open('test_indices_rapids.pkl', 'wb') as f:
    pickle.dump(test_indices, f)
print("\n✓ Test indices saved for evaluation")

## Define Model Evaluation Functions (CPU & GPU)

In [ ]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """
    Train and evaluate a sklearn model on CPU
    Returns: dictionary with model performance metrics
    """
    print(f"\n{'='*80}")
    print(f"Training: {model_name}")
    print(f"{'='*80}")
    
    # Training
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # Print results
    print(f"\n📊 Training Results:")
    print(f"   Training Time: {training_time:.2f} seconds")
    print(f"\n   Training Set:")
    print(f"   - RMSE: {train_rmse:.4f}")
    print(f"   - MAE:  {train_mae:.4f}")
    print(f"   - R²:   {train_r2:.4f}")
    print(f"\n   Test Set:")
    print(f"   - RMSE: {test_rmse:.4f}")
    print(f"   - MAE:  {test_mae:.4f}")
    print(f"   - R²:   {test_r2:.4f}")
    
    # Check for overfitting
    if train_r2 - test_r2 > 0.1:
        print(f"\n⚠️  Warning: Possible overfitting detected (R² difference: {train_r2 - test_r2:.4f})")
    
    # Return results
    return {
        'model_name': model_name,
        'model': model,
        'training_time': training_time,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'y_train_pred': y_train_pred,
        'y_test_pred': y_test_pred
    }

print("✓ CPU evaluation function defined")

In [ ]:
def evaluate_model_gpu(model, X_train, X_test, y_train, y_test, model_name):
    """
    Train and evaluate a cuML model on GPU
    Returns: dictionary with model performance metrics
    """
    print(f"\n{'='*80}")
    print(f"Training: {model_name}")
    print(f"{'='*80}")
    
    # Training
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    # Predictions (on GPU)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate metrics (some on GPU, some on CPU)
    # cuML metrics (GPU)
    train_rmse = cp.sqrt(cu_mse(y_train, y_train_pred))
    test_rmse = cp.sqrt(cu_mse(y_test, y_test_pred))
    train_r2 = cu_r2_score(y_train, y_train_pred)
    test_r2 = cu_r2_score(y_test, y_test_pred)
    
    # MAE not in cuML yet, use numpy (transfer to CPU)
    train_mae = mean_absolute_error(
        y_train.to_numpy() if hasattr(y_train, 'to_numpy') else cp.asnumpy(y_train),
        y_train_pred.to_numpy() if hasattr(y_train_pred, 'to_numpy') else cp.asnumpy(y_train_pred)
    )
    test_mae = mean_absolute_error(
        y_test.to_numpy() if hasattr(y_test, 'to_numpy') else cp.asnumpy(y_test),
        y_test_pred.to_numpy() if hasattr(y_test_pred, 'to_numpy') else cp.asnumpy(y_test_pred)
    )
    
    # Print results
    print(f"\n📊 Training Results:")
    print(f"   Training Time: {training_time:.2f} seconds")
    print(f"\n   Training Set:")
    print(f"   - RMSE: {float(train_rmse):.4f}")
    print(f"   - MAE:  {train_mae:.4f}")
    print(f"   - R²:   {float(train_r2):.4f}")
    print(f"\n   Test Set:")
    print(f"   - RMSE: {float(test_rmse):.4f}")
    print(f"   - MAE:  {test_mae:.4f}")
    print(f"   - R²:   {float(test_r2):.4f}")
    
    # Return results
    return {
        'model_name': model_name,
        'model': model,
        'training_time': training_time,
        'train_rmse': float(train_rmse),
        'test_rmse': float(test_rmse),
        'train_mae': train_mae,
        'test_mae': test_mae,
        'train_r2': float(train_r2),
        'test_r2': float(test_r2),
        'y_train_pred': y_train_pred,
        'y_test_pred': y_test_pred
    }

print("✓ GPU evaluation function defined")

## 1️⃣ Linear Regression (cuML GPU)

In [ ]:
# Linear Regression with cuML (GPU-accelerated)
lr_model = cuLinearRegression(
    fit_intercept=True,
    algorithm='eig'  # 'eig' or 'svd'
)

lr_results = evaluate_model_gpu(
    lr_model, X_train, X_test, y_train, y_test,
    'Linear Regression (cuML GPU)'
)

# Save model
with open('model_linear_regression_rapids.pkl', 'wb') as f:
    pickle.dump(lr_model, f)
print("\n✓ Model saved: model_linear_regression_rapids.pkl")

## 2️⃣ Ridge Regression (cuML GPU)

In [ ]:
# Ridge Regression with cuML (GPU-accelerated)
ridge_model = cuRidge(
    alpha=1.0,
    fit_intercept=True,
    solver='eig'  # 'eig' or 'svd'
)

ridge_results = evaluate_model_gpu(
    ridge_model, X_train, X_test, y_train, y_test,
    'Ridge Regression (cuML GPU)'
)

# Save model
with open('model_ridge_rapids.pkl', 'wb') as f:
    pickle.dump(ridge_model, f)
print("\n✓ Model saved: model_ridge_rapids.pkl")

## 3️⃣ Lasso Regression (cuML GPU)

In [ ]:
# Lasso Regression with cuML (GPU-accelerated)
lasso_model = cuLasso(
    alpha=1.0,
    fit_intercept=True,
    max_iter=1000,
    tol=1e-4
)

lasso_results = evaluate_model_gpu(
    lasso_model, X_train, X_test, y_train, y_test,
    'Lasso Regression (cuML GPU)'
)

# Save model
with open('model_lasso_rapids.pkl', 'wb') as f:
    pickle.dump(lasso_model, f)
print("\n✓ Model saved: model_lasso_rapids.pkl")

## 4️⃣ ElasticNet Regression (cuML GPU)

In [ ]:
# ElasticNet with cuML (GPU-accelerated)
elasticnet_model = cuElasticNet(
    alpha=1.0,
    l1_ratio=0.5,
    fit_intercept=True,
    max_iter=1000,
    tol=1e-4
)

elasticnet_results = evaluate_model_gpu(
    elasticnet_model, X_train, X_test, y_train, y_test,
    'ElasticNet (cuML GPU)'
)

# Save model
with open('model_elasticnet_rapids.pkl', 'wb') as f:
    pickle.dump(elasticnet_model, f)
print("\n✓ Model saved: model_elasticnet_rapids.pkl")

## 5️⃣ Decision Tree Regressor (CPU - sklearn)

In [ ]:
# Decision Tree Regressor (CPU - no GPU version in cuML)
print("\n⚠️  Decision Tree: Using sklearn (CPU) - no GPU implementation available")

# Convert cuDF to pandas for sklearn
X_train_cpu = X_train.to_pandas() if hasattr(X_train, 'to_pandas') else X_train
X_test_cpu = X_test.to_pandas() if hasattr(X_test, 'to_pandas') else X_test
y_train_cpu = y_train.to_pandas() if hasattr(y_train, 'to_pandas') else y_train
y_test_cpu = y_test.to_pandas() if hasattr(y_test, 'to_pandas') else y_test

dt_model = DecisionTreeRegressor(
    max_depth=10,
    min_samples_split=20,
    random_state=42
)

dt_results = evaluate_model(
    dt_model, X_train_cpu, X_test_cpu, y_train_cpu, y_test_cpu,
    'Decision Tree Regressor (sklearn CPU)'
)

# Save model
with open('model_decision_tree_rapids.pkl', 'wb') as f:
    pickle.dump(dt_model, f)
print("\n✓ Model saved: model_decision_tree_rapids.pkl")

## 6️⃣ Random Forest Regressor (cuML GPU)

In [ ]:
# Random Forest with cuML (GPU-accelerated)
rf_model = cuRandomForestRegressor(
    n_estimators=100,
    max_depth=16,
    max_features='sqrt',
    n_bins=128,
    min_samples_split=2,
    random_state=42,
    n_streams=4  # GPU-specific: number of parallel streams
)

rf_results = evaluate_model_gpu(
    rf_model, X_train, X_test, y_train, y_test,
    'Random Forest (cuML GPU)'
)

# Save model
with open('model_random_forest_rapids.pkl', 'wb') as f:
    pickle.dump(rf_model, f)
print("\n✓ Model saved: model_random_forest_rapids.pkl")

## 7️⃣ Gradient Boosting Regressor (CPU - sklearn)

In [ ]:
# Gradient Boosting Regressor (CPU - no GPU version in cuML)
print("\n⚠️  Gradient Boosting: Using sklearn (CPU) - no GPU implementation available")

# Convert cuDF to pandas for sklearn
X_train_cpu = X_train.to_pandas() if hasattr(X_train, 'to_pandas') else X_train
X_test_cpu = X_test.to_pandas() if hasattr(X_test, 'to_pandas') else X_test
y_train_cpu = y_train.to_pandas() if hasattr(y_train, 'to_pandas') else y_train
y_test_cpu = y_test.to_pandas() if hasattr(y_test, 'to_pandas') else y_test

gb_model = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)

gb_results = evaluate_model(
    gb_model, X_train_cpu, X_test_cpu, y_train_cpu, y_test_cpu,
    'Gradient Boosting Regressor (sklearn CPU)'
)

# Save model
with open('model_gradient_boosting_rapids.pkl', 'wb') as f:
    pickle.dump(gb_model, f)
print("\n✓ Model saved: model_gradient_boosting_rapids.pkl")

## 🔟 K-Nearest Neighbors Regressor (cuML GPU)

In [ ]:
# KNN with cuML (GPU-accelerated)
knn_model = cuKNeighborsRegressor(
    n_neighbors=5,
    algorithm='brute',  # 'brute' uses GPU efficiently
    metric='euclidean'
)

knn_results = evaluate_model_gpu(
    knn_model, X_train, X_test, y_train, y_test,
    'K-Nearest Neighbors (cuML GPU)'
)

# Save model
with open('model_knn_rapids.pkl', 'wb') as f:
    pickle.dump(knn_model, f)
print("\n✓ Model saved: model_knn_rapids.pkl")

## 8️⃣ XGBoost with GPU Acceleration

In [ ]:
if xgboost_available:
    # XGBoost with GPU acceleration
    # Convert cuDF to DMatrix for XGBoost
    dtrain = xgb.DMatrix(
        X_train.to_pandas() if hasattr(X_train, 'to_pandas') else X_train,
        label=y_train.to_pandas() if hasattr(y_train, 'to_pandas') else y_train
    )
    dtest = xgb.DMatrix(
        X_test.to_pandas() if hasattr(X_test, 'to_pandas') else X_test,
        label=y_test.to_pandas() if hasattr(y_test, 'to_pandas') else y_test
    )
    
    print(f"\n{'='*80}")
    print(f"Training: XGBoost (GPU Accelerated)")
    print(f"{'='*80}")
    
    params = {
        'tree_method': 'gpu_hist',  # GPU acceleration!
        'predictor': 'gpu_predictor',
        'objective': 'reg:squarederror',
        'max_depth': 6,
        'learning_rate': 0.1,
        'n_estimators': 100,
        'random_state': 42
    }
    
    start_time = time.time()
    xgb_model = xgb.train(
        params,
        dtrain,
        num_boost_round=100,
        evals=[(dtrain, 'train'), (dtest, 'test')],
        verbose_eval=20
    )
    training_time = time.time() - start_time
    
    # Predictions
    y_train_pred_xgb = xgb_model.predict(dtrain)
    y_test_pred_xgb = xgb_model.predict(dtest)
    
    # Metrics
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    train_rmse = np.sqrt(mean_squared_error(y_train.to_pandas(), y_train_pred_xgb))
    test_rmse = np.sqrt(mean_squared_error(y_test.to_pandas(), y_test_pred_xgb))
    train_mae = mean_absolute_error(y_train.to_pandas(), y_train_pred_xgb)
    test_mae = mean_absolute_error(y_test.to_pandas(), y_test_pred_xgb)
    train_r2 = r2_score(y_train.to_pandas(), y_train_pred_xgb)
    test_r2 = r2_score(y_test.to_pandas(), y_test_pred_xgb)
    
    print(f"\n📊 Training Results:")
    print(f"   Training Time: {training_time:.2f} seconds")
    print(f"\n   Test Set:")
    print(f"   - RMSE: {test_rmse:.4f}")
    print(f"   - MAE:  {test_mae:.4f}")
    print(f"   - R²:   {test_r2:.4f}")
    
    xgb_results = {
        'model_name': 'XGBoost (GPU)',
        'model': xgb_model,
        'training_time': training_time,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'y_train_pred': y_train_pred_xgb,
        'y_test_pred': y_test_pred_xgb
    }
    
    # Save model
    xgb_model.save_model('model_xgboost_rapids.json')
    print("\n✓ Model saved: model_xgboost_rapids.json")
else:
    xgb_results = None
    print("⚠ XGBoost not available - skipping")

## 9️⃣ Support Vector Regressor (SVR) - CPU (sklearn)

In [ ]:
# Support Vector Regressor (CPU - no GPU version in cuML)
print("\n⚠️  SVR: Using sklearn (CPU) - no GPU implementation available")
print("⚠️  Note: Training SVR on a subset (50,000 samples) due to computational cost...")

# Convert cuDF to pandas for sklearn
X_train_cpu = X_train.to_pandas() if hasattr(X_train, 'to_pandas') else X_train
X_test_cpu = X_test.to_pandas() if hasattr(X_test, 'to_pandas') else X_test
y_train_cpu = y_train.to_pandas() if hasattr(y_train, 'to_pandas') else y_train
y_test_cpu = y_test.to_pandas() if hasattr(y_test, 'to_pandas') else y_test

# Use subset for SVR
subset_size = min(50000, len(X_train_cpu))
X_train_subset = X_train_cpu.iloc[:subset_size]
y_train_subset = y_train_cpu.iloc[:subset_size]

svr_model = SVR(
    kernel='rbf',
    C=10,
    gamma='scale'
)

svr_results = evaluate_model(
    svr_model, X_train_subset, X_test_cpu, y_train_subset, y_test_cpu,
    'Support Vector Regressor (SVR) - sklearn CPU'
)

# Save model
with open('model_svr_rapids.pkl', 'wb') as f:
    pickle.dump(svr_model, f)
print("\n✓ Model saved: model_svr_rapids.pkl")

## 📊 Compare All Models (GPU Training Results)

In [ ]:
# Collect all results
all_results = [
    lr_results,
    ridge_results,
    lasso_results,
    elasticnet_results,
    dt_results,
    rf_results,
    gb_results,
    svr_results,
    knn_results
]

if xgboost_available and xgb_results:
    all_results.insert(7, xgb_results)  # Insert XGBoost after GB, before SVR

# Create comparison DataFrame
comparison_df = pd.DataFrame([
    {
        'Model': r['model_name'],
        'Training Time (s)': r['training_time'],
        'Test RMSE': r['test_rmse'],
        'Test MAE': r['test_mae'],
        'Test R²': r['test_r2']
    }
    for r in all_results
])

comparison_df = comparison_df.sort_values('Test RMSE', ascending=True)

print("\n" + "="*80)
print("MODEL COMPARISON - GPU TRAINING RESULTS")
print("="*80)
print(comparison_df.to_string(index=False))

# Save comparison
comparison_df.to_csv('model_results_rapids.csv', index=False)
print("\n✓ Results saved: model_results_rapids.csv")

## 📈 Visualize Model Comparison

In [ ]:
# Visualize model performance
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Test RMSE Comparison
axes[0, 0].barh(comparison_df['Model'], comparison_df['Test RMSE'], color='coral', alpha=0.7)
axes[0, 0].set_xlabel('RMSE (Lower is Better)', fontweight='bold')
axes[0, 0].set_title('Test RMSE - GPU Trained Models', fontsize=14, fontweight='bold')
axes[0, 0].invert_yaxis()
axes[0, 0].grid(alpha=0.3, axis='x')

# 2. Test MAE Comparison
axes[0, 1].barh(comparison_df['Model'], comparison_df['Test MAE'], color='skyblue', alpha=0.7)
axes[0, 1].set_xlabel('MAE (Lower is Better)', fontweight='bold')
axes[0, 1].set_title('Test MAE - GPU Trained Models', fontsize=14, fontweight='bold')
axes[0, 1].invert_yaxis()
axes[0, 1].grid(alpha=0.3, axis='x')

# 3. Test R² Comparison
axes[1, 0].barh(comparison_df['Model'], comparison_df['Test R²'], color='lightgreen', alpha=0.7)
axes[1, 0].set_xlabel('R² Score (Higher is Better)', fontweight='bold')
axes[1, 0].set_title('Test R² - GPU Trained Models', fontsize=14, fontweight='bold')
axes[1, 0].invert_yaxis()
axes[1, 0].grid(alpha=0.3, axis='x')

# 4. Training Time Comparison
axes[1, 1].barh(comparison_df['Model'], comparison_df['Training Time (s)'], color='gold', alpha=0.7)
axes[1, 1].set_xlabel('Training Time (seconds)', fontweight='bold')
axes[1, 1].set_title('GPU Training Time Comparison', fontsize=14, fontweight='bold')
axes[1, 1].invert_yaxis()
axes[1, 1].grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('model_comparison_rapids.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Visualization saved: model_comparison_rapids.png")

## 💾 Save All Results for Evaluation

In [ ]:
# Save all results for evaluation notebook
with open('all_model_results_rapids.pkl', 'wb') as f:
    pickle.dump(all_results, f)

print("✓ All model results saved: all_model_results_rapids.pkl")
print("\n📁 Generated files:")
print("   Models:")
for result in all_results:
    model_name = result['model_name'].lower().replace(' ', '_').replace('(', '').replace(')', '')
    if 'xgboost' in model_name:
        print(f"   • model_{model_name.split()[0]}_rapids.json")
    else:
        print(f"   • model_{model_name.split()[0]}_rapids.pkl")
print("\n   Results:")
print("   • all_model_results_rapids.pkl")
print("   • model_results_rapids.csv")
print("   • model_comparison_rapids.png")
print("   • test_indices_rapids.pkl")

## 🎯 GPU Training Summary

In [ ]:
print("="*80)
print("GPU-ACCELERATED MODEL TRAINING COMPLETE")
print("="*80)

print(f"\n✅ TRAINED {len(all_results)} MODELS ON GPU:")
for i, result in enumerate(all_results, 1):
    print(f"   {i}. {result['model_name']}")
    print(f"      - Test RMSE: {result['test_rmse']:.4f}")
    print(f"      - Test R²: {result['test_r2']:.4f}")
    print(f"      - Training Time: {result['training_time']:.2f}s")

best_model = min(all_results, key=lambda x: x['test_rmse'])
print(f"\n🏆 BEST MODEL (Lowest Test RMSE):")
print(f"   {best_model['model_name']}")
print(f"   - Test RMSE: {best_model['test_rmse']:.4f}")
print(f"   - Test MAE: {best_model['test_mae']:.4f}")
print(f"   - Test R²: {best_model['test_r2']:.4f}")
print(f"   - Training Time: {best_model['training_time']:.2f}s")

print("\n🚀 GPU ACCELERATION BENEFITS:")
print("   • Linear models: 10-50x faster than scikit-learn")
print("   • Random Forest: 10-25x faster training")
print("   • KNN: Efficient GPU distance calculations")
print("   • XGBoost: Native GPU histogram algorithm")

print("\n💡 NEXT STEPS:")
print("   1. Use Model_Evaluation_RAPIDS.ipynb for detailed analysis")
print("   2. Compare GPU vs CPU training times")
print("   3. Perform GPU-accelerated hyperparameter tuning")
print("   4. Deploy models with GPU inference")

print("\n🎵 APPLICATION:")
print("   GPU-powered prediction of song release years - FAST!")

print("\n" + "="*80)

# Report GPU memory usage
mempool = cp.get_default_memory_pool()
print(f"\n📊 GPU Memory Usage:")
print(f"   Used: {mempool.used_bytes() / 1024**2:.2f} MB")
print(f"   Total: {mempool.total_bytes() / 1024**2:.2f} MB")